In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**READ AN AUDIO FILE**

In [2]:
audio_file_wav = '/content/drive/Shareddrives/298A/audios/do-you-think-we-can-avoid-accidents.wav'

In [3]:
!pip install --upgrade google-cloud-speech

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.9/273.9 kB 27.3 MB/s eta 0:00:00


In [4]:
import os
from google.cloud import speech

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/drive/Shareddrives/298A/speech-to-text-382212-46fc5777a765.json'
speech_client = speech.SpeechClient()

with open(audio_file_wav, 'rb') as f1:
    byte_data = f1.read()
audio_wav = speech.RecognitionAudio(content = byte_data)

config_wav = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    audio_channel_count=2,
    enable_automatic_punctuation=True,
    language_code = 'en-US'
)

response_standard_wav = speech_client.recognize(
    config=config_wav,
    audio=audio_wav
    )

In [5]:
audio_to_text = response_standard_wav.results[0].alternatives[0].transcript

In [6]:
print(audio_to_text)

Do you think we can avoid accidents?


**CONVERT TEXT TO GLOSS**

In [7]:
'''audio_to_text = 'sentence' '''

"audio_to_text = 'sentence' "

In [8]:
# Install the most re version of TensorFlow to use the improved
# masking support for `tf.keras.layers.MultiHeadAttention`.
!apt install --allow-change-held-packages libcudnn8=8.6.0.163-1+cuda11.8
!pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
!pip install protobuf~=3.20.3
!pip install -q tensorflow_datasets
!pip install -q -U tensorflow-text tensorflow
!pip install transformers --quiet
# Installing the ffmpeg
!pip install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be DOWNGRADED:
  libcudnn8
0 upgraded, 0 newly installed, 1 downgraded, 1 to remove and 22 not upgraded.
Need to get 446 MB of archives.
After this operation, 1,441 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  libcudnn8 8.6.0.163-1+cuda11.8 [446 MB]
Fetched 446 MB in 5s (91.6 MB/s)
(Reading database ... 122531 files and directories currently installed.)
Removing libcudnn8-dev (8.7.0.84-1+cuda11.8) ...
update-alternatives: removing manually selected alternative - switching libcudnn to auto mode
dpkg: warning: downgrading libcudnn8 from 8.7.0.84-1+cuda11.8 to 8.6.0.163-1+cuda11.8
(Reading database ... 122498 files and directories currently installed.)
Preparing to unpack .../libcudnn8_8.6.0

In [9]:
import logging
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow as tf
import tensorflow_text
from tensorflow.data import AUTOTUNE
import collections
import os
import pathlib
import re
import string
import sys
import tempfile

In [10]:
%cd /content/drive/Shareddrives/298A/Transformer_Inference

/content/drive/Shareddrives/298A/Transformer_Inference


In [11]:
tokenizer = tf.saved_model.load('tx_gl_converter')
tokenizer.tx.get_vocab_size().numpy()

4801

In [12]:
tx_gl_translation = tf.saved_model.load('tx-gl-translator')

In [13]:
gloss_text = tx_gl_translation(audio_to_text).numpy()

In [14]:
import re

# Define the pattern to match special characters
pattern = r'[^a-zA-Z0-9\s]'
# Use regular expression substitution to remove special characters
cleaned_string = re.sub(pattern, '', audio_to_text)

converted_sentence = gloss_text.decode('utf-8')  # Convert bytes to string
converted_sentence = converted_sentence.replace('x', '')  # Replace 'x' with 'you'
converted_sentence = converted_sentence.replace('desc', '')  # Replace 'desc' with 'you'
converted_sentence = converted_sentence.replace('-', '')  # Replace '-' with ''
converted_sentence = converted_sentence.replace('  ', '')  # Replace double space with single space

gloss_text = converted_sentence

In [15]:
print(gloss_text)

do you think we can avoid accident ?


**GLOSS TO SKELETON VIDEO THROUGH LOOKUP**

In [16]:
!pwd

/content/drive/Shareddrives/298A/Transformer_Inference


In [17]:
%cd /content/drive/Shareddrives/298A

/content/drive/Shareddrives/298A


In [18]:
# Reading the Video Metadata
import pandas as pd
meta_df = pd.read_csv("video-metadata.csv")

In [19]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')  # Download the WordNet corpus

tokens = nltk.word_tokenize(gloss_text)
lemmatizer = WordNetLemmatizer()

gloss_vals = []
# Iterate over each token
for token in tokens:
  lemma = lemmatizer.lemmatize(token, pos='v')
  gloss_vals.append(lemma)

gloss_text = " ".join(gloss_vals)
print(gloss_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


do you think we can avoid accident ?


In [20]:
lemma

'?'

In [21]:
# Defining the function to extract gloss from the gloss sequence
def get_pose_ids(gloss):
  gloss_list = gloss.split(" ")
  print(gloss_list)
  pose_ids = list()
  meta_df["Gloss Variant"] = meta_df["Gloss Variant"].apply(lambda x: x.strip("+#_()123456789"))
  for term in gloss_list:
    ids = meta_df[meta_df["Gloss Variant"] == term]["id"].values
    if len(ids) > 0:
      pose_ids.append(ids[0])
  return pose_ids

# Fetch the video clip based on the pose id
def get_pose_files(lookup_directory, pose_ids):
    pose_filepaths = []
    for pose_id in pose_ids:
        pose_filename = "pose-{}.mov".format(pose_id)
        pose_filepath = os.path.join(lookup_directory, pose_filename)
        pose_filepaths.append(pose_filepath)
    return pose_filepaths

In [22]:
# Get posses from the gloss by calling the get_pose_ids function
extracted_poses = get_pose_ids(gloss_text.upper())
gloss_available = []
for i in extracted_poses:
  glo = meta_df[meta_df["id"] == i]["Gloss Variant"]
  gloss_available.append(glo)

gloss_available

['DO', 'YOU', 'THINK', 'WE', 'CAN', 'AVOID', 'ACCIDENT', '?']


[3167    DO
 Name: Gloss Variant, dtype: object,
 8547    THINK
 Name: Gloss Variant, dtype: object,
 2007    CAN
 Name: Gloss Variant, dtype: object,
 399    ACCIDENT
 Name: Gloss Variant, dtype: object]

In [23]:
print("Total number of gloss in a gloss sequence: ",len(gloss_text.split(" ")))
print("Number of gloss which found a mapping to video clip: ",len(extracted_poses))

Total number of gloss in a gloss sequence:  8
Number of gloss which found a mapping to video clip:  4


In [24]:
# Fetching the video clips filepaths
video_filenames = get_pose_files('lookup/', extracted_poses)

In [25]:
# creating a .mp4 file from .mov
import os.path
from os import path

video_filenames_mp4 = []
for every_video in video_filenames:
  input_video = every_video
  output_video = 'lookup_gloss_videolist/lookup_mp4/'+input_video.split('/')[1].split('.')[0]+'.mp4'
  video_filenames_mp4.append('lookup_mp4/'+input_video.split('/')[1].split('.')[0]+'.mp4')
  if path.exists(output_video) == False:
    print("convertion to mp4 is required for: ",input_video)
    !ffmpeg -i $input_video -b:v 1M -vcodec libx264 -preset slow -profile:v baseline -level 3.0 -pix_fmt yuv420p -movflags +faststart $output_video

In [26]:
# Creating the content to merge the required video files
updated_gloss = gloss_text.upper().replace(" ","_")
gloss_videolist = "lookup_gloss_videolist/"+updated_gloss+".txt"
gloss_videolist_to_write = "file 'lookup_gloss_videolist/"+updated_gloss+".txt'"

#print(updated_gloss, gloss_videolist, gloss_videolist_to_write)
if path.exists(gloss_videolist) == False:
   with open(gloss_videolist, "w") as f:
      # Write each value in the list as a new line in the file
      for item in video_filenames_mp4:
        f.write("file '" + item + "'\n")

In [27]:
# Check by reading the file in read mode
with open(gloss_videolist, "r") as f:
    file_content = f.read()
    print(file_content)

file 'lookup_mp4/pose-3167.mp4'
file 'lookup_mp4/pose-8547.mp4'
file 'lookup_mp4/pose-2007.mp4'
file 'lookup_mp4/pose-399.mp4'



In [28]:
updated_gloss

'DO_YOU_THINK_WE_CAN_AVOID_ACCIDENT_?'

In [29]:
lst = ['.mp4']
for ext in lst:
  file_path = os.path.join("/content/drive/Shareddrives/298A/lookup_gloss_videolist", updated_gloss + ext)
  print(file_path)
  try:
      if os.path.isfile(file_path) or os.path.islink(file_path):
          os.unlink(file_path)
      elif os.path.isdir(file_path):
          shutil.rmtree(file_path)
  except Exception as e:
      print('Failed to delete %s. Reason: %s' % (file_path, e))

/content/drive/Shareddrives/298A/lookup_gloss_videolist/DO_YOU_THINK_WE_CAN_AVOID_ACCIDENT_?.mp4


In [30]:
#Merging all the relavant videos
merged_input = gloss_videolist
merged_output = gloss_videolist.split(".")[0]+".mp4"
#print(merged_input, merged_output)
!ffmpeg -f concat -i $merged_input -c copy $merged_output

print(merged_input)
print(merged_output)

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [31]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open(merged_output,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)